In [2]:
%matplotlib inline
#%load_ext autoreload
#%autoreload 2
%reload_ext autoreload
import numpy as np
import matplotlib.pyplot as plt
import math, sys, os
from numpy.random import randn
from sklearn.datasets import make_blobs

# setup pyspark for IPython_notebooks
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.10 (default, Jul 13 2015 12:05:58)
SparkContext available as sc, HiveContext available as sqlContext.


In [9]:
data_home = os.environ.get('DATA_HOME', None)
sys.path.insert(0, data_home)

# data
from gen_data import make_blobs_rdd

# utilitiy functions for this notebook
from lsh_util import *

# make some data
N = 1000
d = 2
k = 5
sigma = 1
bound = 10*sigma

data_RDD = make_blobs_rdd(N, d, k, sigma, bound, sc)
data_RDD.take(2)

[(0, array([ 0.17136833,  0.46758811])),
 (2, array([-0.85104408,  1.26787849]))]

In [16]:
def config_stable_dist_proj(d, p = 5, r = 2.0, seed = None):
    # random projection vectors
    A = np.random.multivariate_normal(np.zeros(d), np.eye(d), p)
    B = np.random.rand(1,p)
    def projection_func(tup):
        y, x = tup # expect key (int, 1xD vector)
        projs = ((A.dot(x) / r) + B).flatten()
        bucket = to_bucket(projs)
        return (bucket, y)
    return (A, B, projection_func)

In [17]:
A, B, hash_func = config_stable_dist_proj(d)

gini_impurities = data_RDD.map(hash_func).map(to_dict).reduceByKey(reduce_count_clusters).map(gini_impurity).collect()
for b, g, c in sorted(gini_impurities):
    print "bucket: %s , in bucket: %d , gini_impurity: %f" % (b, c, g)


bucket: 22 , in bucket: 4 , gini_impurity: 0.000000
bucket: 23 , in bucket: 205 , gini_impurity: 0.143914
bucket: 26 , in bucket: 35 , gini_impurity: 0.107755
bucket: 27 , in bucket: 212 , gini_impurity: 0.123354
bucket: 30 , in bucket: 167 , gini_impurity: 0.000000
bucket: 31 , in bucket: 377 , gini_impurity: 0.527774
